# Mount google drive to get data
(just for Marit)

In [24]:
# import os
# from google.colab import drive
# drive.mount('/content/gdrive');
# os.chdir('gdrive/My Drive/Information Retrieval/Information Retrieval Project/Notebooks')
# os.listdir()

## Install not common libraries

In [25]:
# !pip install janome
# !pip install gensim
# !pip install nltk
# !pip install tqdm
# !pip3 install stemming
# !pip install rank_bm25

## Import libraries

In [26]:
import numpy as np
import pandas as pd
# from IPython.display import clear_output
from tqdm import tqdm
# import re
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
nltk.download('wordnet')
from rank_bm25 import BM25Okapi


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sjoerd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess text methods

In [27]:
stopwords = []
stop_words_path = '..\\Data\\nfcorpus\\raw\\stopwords.large'
stop_words_path = '../Data/nfcorpus/raw/stopwords.large'
# create list of stop words in stopword document
with open(stop_words_path, encoding='utf-8') as f:
    for l in f:
        if l[0] in ('\n', '#'):
            continue
        stopwords.append(l.rstrip())

In [28]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return SnowballStemmer("english").stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stopwords and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result


In [29]:
# test preprocess actions
test_sentence = "test string walking the beaches study"
print(test_sentence)
print(preprocess(test_sentence))


test string walking the beaches study
['test', 'string', 'walk', 'beach', 'studi']


In [30]:
# load the raw query documents
raw_dataset = pd.read_csv("../Data/nfcorpus/raw/nfdump.txt", sep='\t', index_col=False, header=None )
clean_queries = raw_dataset#[[0, 5]]
# Column names from the readme
column_names = {
    0: "ID",
    1 : "URL",
    2 : "TITLE",
    3 : "MAINTEXT",
    4 : "COMMENTS",
    5 : "TOPICS_TAGS",
    6 : "DESCRIPTION",
    7 : "DOCTORS_NOTE",
    8 : "ARTICLE_LINKS",
    9 : "QUESTION_LINKS", 
    10: "TOPIC_LINKS", 
    11: "VIDEO_LINKS",
    12: "MEDARTICLE_LINKS"
}
## Add column names to columns that need to be referenced later 
clean_queries = clean_queries.rename(columns=column_names)
# Delete all advertisements
Ad_indices = clean_queries[(clean_queries.TOPICS_TAGS == 'DVD') | (clean_queries.TOPICS_TAGS == 'jobs')].index
clean_queries = clean_queries.drop(Ad_indices)

# Set indices to document identifiers
clean_queries = clean_queries.set_index('ID')

# Reformat topic string to topic array
clean_queries['TOPICS_TAGS'] = clean_queries.TOPICS_TAGS.apply(lambda x: x.split(','))
## Stopword removal, Lemmatize and stemming of the title and maintext
clean_queries.TITLE = clean_queries.TITLE.apply(preprocess)
clean_queries.MAINTEXT = clean_queries.MAINTEXT.apply(preprocess)
preprocess
print(f"The columns of the raw dataset are {clean_queries.columns.values}")

The columns of the raw dataset are ['URL' 'TITLE' 'MAINTEXT' 'COMMENTS' 'TOPICS_TAGS' 'DESCRIPTION'
 'DOCTORS_NOTE' 'ARTICLE_LINKS' 'QUESTION_LINKS' 'TOPIC_LINKS'
 'VIDEO_LINKS' 'MEDARTICLE_LINKS']


In [31]:
#Preview dataset
clean_queries.head(5)

,URL,TITLE,MAINTEXT,COMMENTS,TOPICS_TAGS,DESCRIPTION,DOCTORS_NOTE,ARTICLE_LINKS,QUESTION_LINKS,TOPIC_LINKS,VIDEO_LINKS,MEDARTICLE_LINKS
ID,,,,,,,,,,,,
PLAIN-1,http://nutritionfacts.org/2015/07/21/why-deep-...,"[deep, fri, food, cancer]","[latest, studi, dietari, pattern, breast, canc...","Even before the oil gets in the fryer, it’s be...","[acrylamide, beans, breast cancer, cabbage, ca...",-,-,-,-,http://nutritionfacts.org/topics/organic-foods...,http://nutritionfacts.org/video/meat-fumes-die...,"http://www.ncbi.nlm.nih.gov/pubmed/23092936,ht..."
PLAIN-2,http://nutritionfacts.org/2015/07/16/do-choles...,"[cholesterol, statin, drug, breast, cancer]","[breast, cancer, cholesterol, potenti, mechan,...","Off topic, but what does the science say about...","[breast cancer, cancer, cancer survival, cardi...",-,-,-,-,http://nutritionfacts.org/topics/organic-foods...,http://nutritionfacts.org/video/statin-cholest...,"http://www.ncbi.nlm.nih.gov/pubmed/16565487,ht..."
PLAIN-3,http://nutritionfacts.org/2015/07/14/breast-ca...,"[breast, cancer, cell, fee, cholesterol]","[american, women, diagnos, breast, cancer, lif...",When I eat fresh stalks of celery my mouth goe...,"[body fat, breast cancer, breast cancer surviv...",-,-,-,-,http://nutritionfacts.org/topics/organic-foods...,http://nutritionfacts.org/video/flaxseeds-brea...,"http://www.ncbi.nlm.nih.gov/pubmed/3081176,htt..."
PLAIN-4,http://nutritionfacts.org/2015/07/09/using-die...,"[diet, treat, asthma, eczema]","[previous, discuss, power, fruit, veget, preve...",I am confused about coconut fat. Is it the pro...,"[asthma, blood pressure, caloric restriction, ...",-,-,-,-,http://nutritionfacts.org/topics/organic-foods...,http://nutritionfacts.org/video/uprooting-the-...,"http://www.ncbi.nlm.nih.gov/pubmed/11840688,ht..."
PLAIN-5,http://nutritionfacts.org/2015/07/07/treating-...,"[treat, asthma, plant, pill]","[video, treat, asthma, fruit, veget, highlight...",Maybe the asthma trick is in what not to eat. ...,"[allergies, antioxidants, apples, asthma, bean...",-,-,-,-,http://nutritionfacts.org/topics/organic-foods...,http://nutritionfacts.org/video/alkylphenol-en...,"http://www.ncbi.nlm.nih.gov/pubmed/21623967,ht..."


## Save clean query dataset

##### Save train set

In [32]:
train_indices = np.genfromtxt("../Data/nfcorpus/raw/train.queries.ids", delimiter='\n', encoding="utf8", dtype=None)
print(len(train_indices))
indices = []
for index, a in enumerate(train_indices):
    try:
        clean_queries.loc[a]
    except:
        indices.append(index)

train_indices = np.delete(train_indices, indices)

clean_queries_train = clean_queries.loc[train_indices]

clean_queries_train.to_csv('../Data/clean/query.train.csv')


2594


##### Save dev set

In [33]:
val_indices = np.genfromtxt("../Data/nfcorpus/raw/dev.queries.ids", delimiter='\n', encoding="utf8", dtype=None)
print(len(val_indices))
indices = []
for index, a in enumerate(val_indices):
    try:
        clean_queries.loc[a]
    except:
        indices.append(index)

val_indices = np.delete(val_indices, indices)

clean_queries_val = clean_queries.loc[val_indices]

clean_queries_val.to_csv('../Data/clean/query.dev.csv')


325


##### Save test set

In [34]:
test_indices = np.genfromtxt("../Data/nfcorpus/raw/test.queries.ids", delimiter='\n', encoding="utf8", dtype=None)
print(len(test_indices))
indices = []
for index, a in enumerate(test_indices):
    try:
        clean_queries.loc[a]
    except:
        indices.append(index)

test_indices = np.delete(test_indices, indices)

clean_queries_test = clean_queries.loc[test_indices]

clean_queries_test.to_csv('../Data/clean/query.test.csv')


325


## Load the document dataset

In [35]:
raw_docs_dataset = pd.read_csv("../Data/nfcorpus/raw/doc_dump.txt", sep='\t', index_col=False, header=None )
column_names = { 
    0: "ID",
    1 : "URL", 
    2 : "TITLE",
    3 : "ABSTRACT"
}
raw_docs_dataset = raw_docs_dataset.rename(columns=column_names)

raw_docs_dataset = raw_docs_dataset.set_index('ID')

docs_dataset = raw_docs_dataset.copy()
## Stopword removal, Lemmatize and stemming of the title and abstract
docs_dataset.TITLE = docs_dataset.TITLE.apply(preprocess)
docs_dataset.ABSTRACT = docs_dataset.ABSTRACT.apply(preprocess)



#docs_dataset.TITLE = np.expand_dims(np.squeeze([doc.split(" ") for doc in docs_dataset.TITLE.values]), axis=1)
#docs_dataset.ABSTRACT = ([doc.split(" ") for doc in docs_dataset.ABSTRACT.values])
# Remove standard title beginning from abstract ("Abstract:", "Abstract Background:", "Background research" and "Abstract Background and methods"
#Remove Abstract as first word from the paper
docs_dataset.ABSTRACT = [doc[1:] if "abstract" in doc[0].lower() else doc for doc in docs_dataset.ABSTRACT.values]
# Remove background if it is the first or second word of the paper
#docs_dataset.ABSTRACT = [doc[1:] if "background" in doc[0].lower() else doc for doc in docs_dataset.ABSTRACT.values]
#Remove the first words if an : occured in 
docs_dataset.ABSTRACT = [doc[1 + min([i for i,x in enumerate(doc) if ':' in x]):] if min([i for i,x in enumerate(doc) if ':' in x], default = 999) < 5  else doc for doc in docs_dataset.ABSTRACT.values]

docs_dataset.head(5)

,URL,TITLE,ABSTRACT
ID,,,
MED-1,http://www.ncbi.nlm.nih.gov/pubmed/23092936,"[birth, weight, head, circumfer, prenat, expos...","[background, acrylamid, common, dietari, expos..."
MED-2,http://www.ncbi.nlm.nih.gov/pubmed/22809476,"[statist, regress, model, estim, acrylamid, co...","[human, exposur, acrylamid, consumpt, french, ..."
MED-3,http://www.ncbi.nlm.nih.gov/pubmed/19158207,"[chronic, intak, potato, chip, human, increas,...","[background, high, concentr, acrylamid, common..."
MED-4,http://www.ncbi.nlm.nih.gov/pubmed/23651876,"[dietari, pattern, breast, cancer, risk, women...","[object, breast, cancer, common, type, cancer,..."
MED-5,http://www.ncbi.nlm.nih.gov/pubmed/16332665,"[empir, deriv, dietari, pattern, risk, postmen...","[background, inconsist, associ, report, diet, ..."


In [36]:
#Example result:
preprocessed_abstract = docs_dataset.ABSTRACT.loc['MED-10'][:]

print("Original:\t", raw_docs_dataset.ABSTRACT.loc['MED-10'][:len(' '.join(preprocessed_abstract))])
print("Preprocessed\t", preprocessed_abstract)

Original:	 Abstract Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants died, of which 3,619 (60.2%) was due to breast cancer. Aft
Preprocessed	 ['rec

## Create document train dataset

##### Write one full clean dataset

In [135]:
#Remove all documents without topic
med_indices_filepath = "../Data/clean/med_documents.csv"
med_indices = pd.read_csv(med_indices_filepath,encoding="utf8",header=1, index_col=0)
indices = []
for index, a in enumerate(med_indices.itertuples()):
    if(len(a[1])< 3):
        indices.append(index)

#print(med_indices)
print("To be deleted:", len(indices))
med_indices = np.delete(med_indices.index.values, indices)

docs_dataset = docs_dataset.loc[med_indices]

To be deleted: 283


In [136]:
docs_dataset.to_csv('../Data/clean/doc.all.csv')

# forget the raw dataset
raw_docs_dataset =  None

docs_dataset.head(5)

,URL,TITLE,ABSTRACT
ID,,,
MED-2,http://www.ncbi.nlm.nih.gov/pubmed/22809476,"[statist, regress, model, estim, acrylamid, co...","[human, exposur, acrylamid, consumpt, french, ..."
MED-3,http://www.ncbi.nlm.nih.gov/pubmed/19158207,"[chronic, intak, potato, chip, human, increas,...","[background, high, concentr, acrylamid, common..."
MED-4,http://www.ncbi.nlm.nih.gov/pubmed/23651876,"[dietari, pattern, breast, cancer, risk, women...","[object, breast, cancer, common, type, cancer,..."
MED-5,http://www.ncbi.nlm.nih.gov/pubmed/16332665,"[empir, deriv, dietari, pattern, risk, postmen...","[background, inconsist, associ, report, diet, ..."
MED-6,http://www.ncbi.nlm.nih.gov/pubmed/23335051,"[consumpt, deep, fri, food, risk, prostat, can...","[background, evid, suggest, high, heat, cook, ..."


##### Create the training document dataset

In [38]:
train_indices_filepath = "../Data/nfcorpus/raw/train.docs.ids"
train_indices = np.genfromtxt(train_indices_filepath, delimiter='\n', encoding="utf8", dtype=None)
docs_dataset_train = docs_dataset.loc[train_indices]

##### Create a validation dataset

In [39]:
val_indices_filepath = "../Data/nfcorpus/raw/dev.docs.ids"
val_indices = np.genfromtxt(val_indices_filepath, delimiter='\n', encoding="utf8", dtype=None)
indices = []
for index, a in enumerate(val_indices):
    try:
        docs_dataset.loc[a]
    except:
        indices.append(index)

val_indices = np.delete(val_indices, indices)

docs_dataset_val = docs_dataset.loc[val_indices]

##### Create a test dataset

In [40]:
test_indices_filepath = "../Data/nfcorpus/raw/test.docs.ids"
test_indices = np.genfromtxt(test_indices_filepath, delimiter='\n', encoding="utf8", dtype=None)
indices = []
for index, a in enumerate(test_indices):
    try:
        docs_dataset.loc[a]
    except:
        indices.append(index)

test_indices = np.delete(test_indices, indices)
docs_dataset_test = docs_dataset.loc[test_indices]

In [41]:
docs_dataset.dtypes

URL         object
TITLE       object
ABSTRACT    object
dtype: object

##### Write to disk

In [42]:
docs_dataset_train.to_csv('../Data/clean/doc.train.csv')
docs_dataset_val.to_csv('../Data/clean/doc.dev.csv')
docs_dataset_test.to_csv('../Data/clean/doc.test.csv')



## Create bm25 ranking for all documents

##### Process train queries

In [137]:
bm25 = BM25Okapi((docs_dataset['TITLE'] + docs_dataset['ABSTRACT']))

bm25_ranking = pd.DataFrame(columns = ['query','ranking', 'scores'])
row_index = 0
for row in tqdm(clean_queries.itertuples(), total=len(clean_queries)):
    tokenized_query = row.TITLE + row.MAINTEXT 

    doc_scores = bm25.get_scores(tokenized_query)

    # Get the first 1000 indexes of the sorted list
    # Make array negative to achieve sorting in descending order
    ranking = np.argsort(-doc_scores, axis=0)[:1000]
    top_1000_rank   = docs_dataset.iloc[ranking].index.values
    top_1000_scores = doc_scores[ranking]

    bm25_ranking.loc[row_index] = [row.Index, top_1000_rank.tolist(), top_1000_scores.tolist()]
    row_index += 1

bm25_ranking = bm25_ranking.set_index('query')
# bm25_ranking.to_csv('../Data/clean/ranking.bm25.train2.csv', sep=',')
bm25_ranking.to_csv('../Data/results/ranking.bm25.csv', sep=',')


100%|██████████████████████████████████████████████████████████████████████████████| 3419/3419 [08:58<00:00,  6.35it/s]


In [44]:
bm25_ranking.to_csv('../Data/results/ranking.bm25.csv', sep=',')


In [ ]:
bm25 = BM25Okapi((docs_dataset['TITLE'] + docs_dataset['ABSTRACT']))

bm25_ranking = pd.DataFrame(columns = ['query','ranking', 'scores'])
row_index = 0
for row in tqdm(clean_queries_train.itertuples(), total=len(clean_queries_train)):
    tokenized_query = row.MAINTEXT

    doc_scores = bm25.get_scores(tokenized_query)

    # Get the first 1000 indexes of the sorted list
    # Make array negative to achieve sorting in descending order
    ranking = np.argsort(-doc_scores, axis=0)[:1000]
    top_1000_rank   = docs_dataset.iloc[ranking].index.values
    top_1000_scores = doc_scores[ranking]

    bm25_ranking.loc[row_index] = [row.Index, top_1000_rank.tolist(), top_1000_scores.tolist()]
    row_index += 1

bm25_ranking = bm25_ranking.set_index('query')
# bm25_ranking.to_csv('../Data/clean/ranking.bm25.train2.csv', sep=',')
bm25_ranking.to_csv('../Data/results/ranking.bm25.train.csv', sep=',')


##### Process validation queries

In [ ]:
bm25 = BM25Okapi((docs_dataset['TITLE'] + docs_dataset['ABSTRACT']))

bm25_ranking = pd.DataFrame(columns = ['query','ranking', 'scores'])
row_index = 0
for row in tqdm(clean_queries_val.itertuples(), total=len(clean_queries_val)):
    tokenized_query = row.MAINTEXT

    doc_scores = bm25.get_scores(tokenized_query)

    # get the first 1000 indexes of the sorted list
    # Make array negative to achieve sorting in descending order
    ranking = np.argsort(-doc_scores, axis=0)[:1000]
    top_1000_rank   = docs_dataset.iloc[ranking].index.values
    top_1000_scores = doc_scores[ranking]

    bm25_ranking.loc[row_index] = [row.Index, top_1000_rank.tolist(), top_1000_scores.tolist()]
    row_index += 1

bm25_ranking = bm25_ranking.set_index('query')
# bm25_ranking.to_csv('../Data/clean/ranking.bm25.train2.csv', sep=',')
bm25_ranking.to_csv('../Data/results/ranking.bm25.dev.csv', sep=',')


##### Process test queries

In [ ]:
bm25 = BM25Okapi((docs_dataset['TITLE'] + docs_dataset['ABSTRACT']))

bm25_ranking = pd.DataFrame(columns = ['query','ranking', 'scores'])
row_index = 0
for row in tqdm(clean_queries_test.itertuples(), total=len(clean_queries_test)):
    tokenized_query = row.MAINTEXT

    doc_scores = bm25.get_scores(tokenized_query)

    # get the first 1000 indexes of the sorted list
    # Make array negative to achieve sorting in descending order
    ranking = np.argsort(-doc_scores, axis=0)[:1000]
    top_1000_rank   = docs_dataset.iloc[ranking].index.values
    top_1000_scores = doc_scores[ranking]

    bm25_ranking.loc[row_index] = [row.Index, top_1000_rank.tolist(), top_1000_scores.tolist()]
    row_index += 1

bm25_ranking = bm25_ranking.set_index('query')
# bm25_ranking.to_csv('../Data/clean/ranking.bm25.train2.csv', sep=',')
bm25_ranking.to_csv('../Data/results/ranking.bm25.test.csv', sep=',')


In [ ]:
bm25_ranking